In [89]:
import sys
sys.path.append('helpers/')

from pprint import pprint
import scipy.io as scio
from torch.utils.data import Dataset, DataLoader
from eegdProcessors import getEpochsDataFromRaw
import random

In [90]:
eeg_1s_raw = scio.loadmat('./data/subject_1_eeg_separated_epochs_1s.mat')

In [96]:
def dataToListOfEpochs(data):
    list = []
    nOfEpochs = data.shape[2]
    for i in range(nOfEpochs):
        list.append(data[:,:,i])
    return list

def getRandomNumberOfEpochs(epochs, nofepochs, seed=None):
    epochsLen = len( epochs )

    if nofepochs > epochsLen:
        raise Exception("Broj trazenih epocha veci od broja dostupnih epocha!")
    # Ako je dan seed koristiti ga
    if seed != None:
        random.seed(seed)
    # Randomizira redosljed epocha u listi
    random.shuffle(epochs)
    # Reseedanje random generatora da ostatak sistema ne koristi odabrani seed
    random.seed()
    # Uzima prvih n epocha gjde je n broj trazenih epocha
    return epochs[0:nofepochs]

def getLabeledEpochs(raw_data, ratio, seed=None):
        target_data = getEpochsDataFromRaw(raw_data, 0)
        nonTarget_data = getEpochsDataFromRaw(raw_data, 1)

        target_epochs = dataToListOfEpochs(target_data)
        nonTarget_epochs = dataToListOfEpochs(nonTarget_data)

        lenTarget = len(target_epochs)
        lenNonTarget = len(nonTarget_epochs)

        rndedTargetEphs = []
        rndedNonTargetEphs = []

        if lenTarget/lenNonTarget > ratio:
            # Uzimam sve nonTarget podatke i koliko treba targeta
            rndedNonTargetEphs = getRandomNumberOfEpochs(nonTarget_epochs, lenNonTarget, seed)
            newLenTarget = int(lenNonTarget*ratio)
            rndedTargetEphs = getRandomNumberOfEpochs(target_epochs, newLenTarget, seed)
        elif lenTarget/lenNonTarget < ratio:
            # Uzimam sve target podatke i kolito treba nontargeta
            rndedTargetEphs = getRandomNumberOfEpochs(target_epochs, lenTarget, seed)
            newNonTargetLen = int(lenTarget*(1/ratio))
            rndedNonTargetEphs = getRandomNumberOfEpochs(nonTarget_epochs, newNonTargetLen, seed)
        
        print(len(rndedTargetEphs)/len(rndedNonTargetEphs))
        print('total> ', len(rndedTargetEphs)+len(rndedNonTargetEphs))

In [98]:
getLabeledEpochs(eeg_1s_raw, 3/1)

SETNAME>  EEG target
DATA SHAPE>  (16, 512, 305)
SETNAME>  EEG non-target
DATA SHAPE>  (16, 512, 6447)
3.01980198019802
total>  406


In [ ]:
# Definiranje klase pyTorch dataseta za eeg podatke

class eegDataset(Dataset):
    def __init__(self, eeg_raw, ratio):
        pass    

    def __len__(self):
        return len(self.epochs)